# 🌿 Vertical Garden Impact LSTM — Training Notebook

Train an LSTM model to predict the air-quality impact of a **vertical garden**
installation, based on a 24-step time-series of environmental conditions.

**Inputs (per time-step):** `current_aqi`, `current_pm25`, `area_m2`, `temperature`, `humidity`

**Outputs:** `pm25_reduction`, `pm10_reduction`, `aqi_improvement`, `temperature_reduction`, `noise_reduction`

Saves:
- `models/garden_lstm.pth`
- `models/garden_scaler.json`

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
import json
import os
import matplotlib.pyplot as plt

print(f"PyTorch {torch.__version__}")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

In [ ]:
# ======================== CONFIG ========================
SEQ_LEN      = 24       # 24 time-steps (hours)
INPUT_DIM    = 5        # aqi, pm25, area_m2, temp, humidity
OUTPUT_DIM   = 5        # pm25_red, pm10_red, aqi_imp, temp_red, noise_red
HIDDEN_DIM   = 64
NUM_LAYERS   = 2
DROPOUT      = 0.2
EPOCHS       = 100
BATCH_SIZE   = 32
LR           = 0.001
NUM_SAMPLES  = 5000

In [ ]:
# ===================== DATA GENERATION =====================
def generate_garden_data(num_samples: int):
    """
    Generate synthetic time-series data for vertical garden impact prediction.
    Each sample is a 24-step sequence of environmental readings,
    with the target being the expected impact of a vertical garden.
    Formula mirrors calculations.ts -> estimateGardenImpact().
    """
    np.random.seed(42)
    X, y = [], []

    for _ in range(num_samples):
        base_aqi      = np.random.uniform(50, 400)
        base_pm25     = np.random.uniform(20, 200)
        area_m2       = np.random.uniform(2, 50)   # Garden area (m²)
        base_temp     = np.random.uniform(15, 45)
        base_humidity = np.random.uniform(30, 90)

        seq = []
        for t in range(SEQ_LEN):
            aqi      = base_aqi      + np.random.normal(0, 10) + t * np.random.normal(0, 0.5)
            pm25     = base_pm25     + np.random.normal(0, 5)  + t * np.random.normal(0, 0.2)
            temp     = base_temp     + np.random.normal(0, 2)  + np.sin(t * np.pi / 12) * 3
            humidity = base_humidity + np.random.normal(0, 3)
            seq.append([aqi, pm25, area_m2, temp, humidity])

        X.append(seq)

        # ---------- Target calculation (mirrors calculations.ts) ----------
        current_pm25 = max(seq[-1][1], 1)

        pm25_per_m2    = 0.2 + np.random.random() * 0.3         # 0.2 – 0.5 µg/m³ per m²
        pm25_reduction = pm25_per_m2 * area_m2
        pm10_reduction = pm25_reduction * 1.3

        pm25_pct        = (pm25_reduction / current_pm25) * 100
        aqi_improvement = min(round(2 + pm25_pct * 3), 15)

        temp_reduction  = 1 + np.random.random()                # 1 – 2 °C
        noise_reduction = 3 + np.random.random() * 2            # 3 – 5 dB

        # Environmental modifiers
        temp_factor     = 1.0 + (seq[-1][3] - 25) * 0.008
        humidity_factor = 1.0 + (seq[-1][4] - 60) * 0.004

        pm25_reduction  *= temp_factor * humidity_factor
        pm10_reduction  *= temp_factor * humidity_factor
        temp_reduction  *= (1.0 + area_m2 * 0.01)               # larger garden -> more cooling
        noise_reduction *= (1.0 + area_m2 * 0.005)

        y.append([
            pm25_reduction,
            pm10_reduction,
            float(aqi_improvement),
            temp_reduction,
            noise_reduction,
        ])

    return np.array(X, dtype=np.float32), np.array(y, dtype=np.float32)


X, y = generate_garden_data(NUM_SAMPLES)
print(f"X shape: {X.shape}  |  y shape: {y.shape}")
print(f"Sample target: {y[0]}")

In [ ]:
# =================== NORMALISATION ===================
x_flat = X.reshape(-1, INPUT_DIM)
x_min  = x_flat.min(axis=0)
x_max  = x_flat.max(axis=0)
y_min  = y.min(axis=0)
y_max  = y.max(axis=0)

X_norm = (X - x_min) / (x_max - x_min + 1e-8)
y_norm = (y - y_min) / (y_max - y_min + 1e-8)

# Train / Validation split (80 / 20)
split = int(0.8 * NUM_SAMPLES)
X_train, X_val = X_norm[:split], X_norm[split:]
y_train, y_val = y_norm[:split], y_norm[split:]

train_ds = TensorDataset(torch.FloatTensor(X_train), torch.FloatTensor(y_train))
val_ds   = TensorDataset(torch.FloatTensor(X_val),   torch.FloatTensor(y_val))

train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True)
val_loader   = DataLoader(val_ds,   batch_size=BATCH_SIZE)

print(f"Train: {len(train_ds)} | Val: {len(val_ds)}")

In [ ]:
# ==================== LSTM MODEL ====================
class GardenLSTM(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, num_layers, dropout=0.2):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        self.lstm = nn.LSTM(
            input_dim, hidden_dim, num_layers,
            batch_first=True, dropout=dropout
        )
        self.fc = nn.Sequential(
            nn.Linear(hidden_dim, 32),
            nn.ReLU(),
            nn.Linear(32, output_dim),
        )

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim).to(x.device)
        out, _ = self.lstm(x, (h0, c0))
        return self.fc(out[:, -1, :])


model     = GardenLSTM(INPUT_DIM, HIDDEN_DIM, OUTPUT_DIM, NUM_LAYERS, DROPOUT).to(device)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LR)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=10, factor=0.5)

total_params = sum(p.numel() for p in model.parameters())
print(f"GardenLSTM  |  Parameters: {total_params:,}")
print(model)

In [ ]:
# ==================== TRAINING LOOP ====================
train_losses = []
val_losses   = []

for epoch in range(EPOCHS):
    # --- train ---
    model.train()
    epoch_loss = 0.0
    for xb, yb in train_loader:
        xb, yb = xb.to(device), yb.to(device)
        pred = model(xb)
        loss = criterion(pred, yb)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
    avg_train = epoch_loss / len(train_loader)
    train_losses.append(avg_train)

    # --- validate ---
    model.eval()
    v_loss = 0.0
    with torch.no_grad():
        for xb, yb in val_loader:
            xb, yb = xb.to(device), yb.to(device)
            v_loss += criterion(model(xb), yb).item()
    avg_val = v_loss / len(val_loader)
    val_losses.append(avg_val)
    scheduler.step(avg_val)

    if (epoch + 1) % 10 == 0:
        print(f"Epoch {epoch+1:3d}/{EPOCHS}  Train: {avg_train:.6f}  Val: {avg_val:.6f}")

print("\n✅ Training complete.")

In [ ]:
# ==================== LOSS CURVES ====================
plt.figure(figsize=(10, 5))
plt.plot(train_losses, label="Train Loss")
plt.plot(val_losses,   label="Val Loss")
plt.xlabel("Epoch")
plt.ylabel("MSE Loss")
plt.title("Garden LSTM — Training Progress")
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
# ==================== SAVE MODEL & SCALER ====================
os.makedirs("models", exist_ok=True)

torch.save(model.state_dict(), "models/garden_lstm.pth")

scaler_data = {
    "x_min": x_min.tolist(),
    "x_max": x_max.tolist(),
    "y_min": y_min.tolist(),
    "y_max": y_max.tolist(),
}
with open("models/garden_scaler.json", "w") as f:
    json.dump(scaler_data, f, indent=2)

print("Saved  models/garden_lstm.pth")
print("Saved  models/garden_scaler.json")
print(f"Model size: {os.path.getsize('models/garden_lstm.pth') / 1024:.1f} KB")

In [ ]:
# ==================== QUICK TEST ====================
model.eval()
test_x = torch.FloatTensor(X_norm[:1]).to(device)

with torch.no_grad():
    pred_norm = model(test_x).cpu().numpy()[0]

pred_actual = pred_norm * (y_max - y_min) + y_min
actual      = y[0]

labels = ["pm25_reduction", "pm10_reduction", "aqi_improvement", "temp_reduction", "noise_reduction"]

print("=" * 55)
print(f"{'Metric':<22} {'Predicted':>12} {'Actual':>12}")
print("-" * 55)
for lbl, p, a in zip(labels, pred_actual, actual):
    print(f"{lbl:<22} {p:>12.4f} {a:>12.4f}")
print("=" * 55)

# JSON output example
result_json = {
    "type": "vertical_garden",
    "predictions": {
        lbl: round(float(p), 4) for lbl, p in zip(labels, pred_actual)
    },
}
print("\nJSON output:")
print(json.dumps(result_json, indent=2))